In [1]:
import pandas as pd
import numpy as np
import random
import os.path
from enum import Enum
from IPython.display import display, HTML
from datetime import datetime

from const import *

def display_df(df):
    display(HTML(df.to_html()))

In [2]:
class DataCleaner():
    
    def __init__(self):
        self.dropna_method = 'all'
        self.is_fill_missing = True
    
    def clean(self, raw_data):
        raw_data = raw_data.dropna(how=self.dropna_method)
        if self.is_fill_missing:
            raw_data = self.fill_missing_date_time(raw_data)
        return raw_data
    
    @staticmethod
    def fill_missing_date_time(raw_data):
        count = 0
        cols = list(raw_data.columns)
        cols.remove(DATE_TIME)
        for col in cols:
            for i in raw_data.index:
                element = raw_data.at[i, col]
                if np.isnan(element) or element == 0:
                    missing_data_date = raw_data.at[i, DATE_TIME]
                    ref_data_date = raw_data.at[i-1, DATE_TIME]
                    raw_data.at[i, col] = raw_data.at[i-1, col]
                    count += 1
#                     print('[CLEAN] Copy data ({}, {}) to ({}, {}))'
#                           .format(ref_data_date, col, missing_data_date, col))
        print('[CLEAN] {} data items are filled'.format(count))
    
        return raw_data

In [3]:
class TechIndicatorEngine():
    def __init__(self):
        pass
    
    def compute(self, data, ti_param):
        if self.extract_alphabet(ti_param.name) == TI_CCI:
            cci = CCI(ti_param)
            data = cci.compute(data)
        elif self.extract_alphabet(ti_param.name) == TI_SMA:
            sma = SMA(ti_param)
            data = sma.compute(data)
        else:
            raise ValueError('[ERROR] Unknown technical indicator: {}'.format(ti_param.name))
        
        print('[TechIndicatorEngine] Finished computation of {}'.format(ti_param.name))
        return data
    
    @staticmethod
    def extract_alphabet(s):
        return ''.join(x for x in s if x.isalpha())

In [4]:
class IndicatorParam():
    def __init__(self, name):
        self.name = name
        
    def __eq__(self, other):
        if self.__class__ != other.__class__:
            return False
        return self.__dict__ == other.__dict__


class CCIParam(IndicatorParam):
    def __init__(self, period, coeff):
        super().__init__(TI_CCI)
        self.period = period
        self.coeff = coeff


class SMAParam(IndicatorParam):
    def __init__(self, period, on):
        super().__init__(TI_CCI)
        self.period = period
        self.on = on

In [5]:
class CCI():
    
    def __init__(self, param):
        self.param = param
        
    def compute(self, data):
        
        # typical price
        data['typical'] = data[[HIGH, LOW, CLOSE]].apply(lambda row: self.typical_price(row), axis=1)
        
        # moving average of typical price
        data['MA'] = data['typical'].rolling(window=self.param.period).mean()
        
        # absolute difference between typical price and MA
        data['abs_diff'] = data[['typical', 'MA']].apply(lambda row: abs(row['typical'] - row['MA']), axis=1)
        
        # mean deviation
        data['MD'] = data['abs_diff'].rolling(window=self.param.period).mean()
        
        # CCI
        data[self.param.name] = data[['typical', 'MA', 'MD']].apply(lambda row: self.cci_formula(row), axis=1)
        
        # drop columns for intermediate steps
        data = data.drop(columns=['typical', 'MA', 'abs_diff', 'MD'])
        
        return data
        
    @staticmethod
    def typical_price(row):
        return (row[HIGH] + row[LOW] + row[CLOSE]) / 3
    
    def cci_formula(self, row):
        return (row['typical'] - row['MA']) / (self.param.coeff * row['MD'])


class SMA():
    
    def __init__(self, param):
        self.param = self.param
        
    def compute(self, data):
        
        data[self.param.name] = data[on].rolling(window=self.param.period).mean()
        
        return data

In [6]:
class Rule():
    def __init__(self, name, indicators_param, ref, f):
        self.name = name
        self.indicators_param = indicators_param
        self.ref = ref
        self.f = f

class OpenPositionParam():
    def __init__(self, rules):
        self.rules = rules  # rule: f(data, ref, self.entry_index, current_index) -> boolean

class ClosePositionParam():
    def __init__(self, rules):
        self.rules = rules  # rule : f(data, ref, self.entry_index, current_index) -> boolean


class Strategy():

    def __init__(self, name, open_position_param, close_position_param, position):
        self.name = name
        self.open_position = open_position_param
        self.close_position = close_position_param
        self.position = position
        self.have_position = False
        self.entry_index = None
        
    def check_event(self, data, current_index):       
        if not self.have_position:  # check for open position
            rule_triggered = self.check_rules(data, current_index, self.open_position.rules)
            if rule_triggered:
                self.have_position = True
                self.entry_index = current_index
                return OPEN_POSITION, rule_triggered
            
        else:  # check for close positioin
            rule_triggered = self.check_rules(data, current_index, self.close_position.rules)
            if rule_triggered:
                self.have_position = False
                self.entry_index = None
                return CLOSE_POSITION, rule_triggered
        
        return NO_EVENT, None
    
    def check_rules(self, data, current_index, rules):
        # logical OR on all the rules
        for rule in rules:
            if rule.f(data, rule.ref, self.entry_index, current_index):
                return rule.name
        return False

In [7]:
class BackTest():
    
    def __init__(self, raw_data, strategies):
        
        # class data
        self.data = raw_data
        self.trade_record = pd.DataFrame()
        self.report = pd.DataFrame()
        self.strategies = strategies
        self.all_ti_param = list()
        
        # service
        self.cleaner = DataCleaner()
        self.ti_engine = TechIndicatorEngine()
        
    def initialize(self):
        self.initialize_trade_record()
        self.initialize_report()
        self.initialize_all_ti_param()
        print('[BACKTEST] Initializated')
    
    def initialize_trade_record(self):
        self.trade_record = pd.DataFrame()
        self.trade_record['date_time'] = None
        self.trade_record['strategy_name'] = None
        self.trade_record['position'] = None
        self.trade_record['event'] = None
        self.trade_record['rule'] = None
        self.trade_record['price'] = None
        self.trade_record['trade_return'] = None
        
    def initialize_report(self):
        self.report = pd.DataFrame()
        self.report['strategy_name'] = None
        self.report['occurrence'] = None
        self.report['occ_profit'] = None
        self.report['occ_loss'] = None
        self.report['return_average'] = None
        self.report['return_std'] = None
        
    def initialize_all_ti_param(self):
        for strategy in self.strategies:
            for rule in (strategy.open_position.rules + strategy.close_position.rules):
                for param in rule.indicators_param:
                    
                    if param.name not in self.data.columns:
                        if not self.object_list_contains_object(self.all_ti_param, param):
                            
                            name = self.make_name([i.name for i in self.all_ti_param], param.name)
                            rule.ref[param.name] = name
                            param.name = name
                            self.all_ti_param.append(param)
                        
    @staticmethod
    def object_list_contains_object(obj_list, obj):
        for obj_temp in obj_list:
            if obj_temp == obj:
                return True
        return False
    
    @staticmethod
    def make_name(name_list, target):
        if target not in name_list:
            return target
        i = 2
        while(True):
            new_name = target + '_{}'.format(i)
            if new_name not in name_list:
                return new_name
            i += 1
    
    def data_preprocess(self):
        self.data = self.cleaner.clean(self.data)
        print('[BACKTEST] Finished data pre-process')
        
    def compute_technical_indicator(self):
        for param in self.all_ti_param:
            self.data = self.ti_engine.compute(self.data, param)
        print('[BACKTEST] Finished computation on technical indicators')
        
    def back_test(self):
        for i in self.data.index:
            for strategy in self.strategies:
                event, rule = strategy.check_event(self.data, i)
                if event is not NO_EVENT:
                    date_time = self.data.at[i, DATE_TIME]
                    price = self.data.at[i, CLOSE]
                    self.record_event(date_time, strategy, event, rule, price)
        self.calculate_return()
        print('[BACKTEST] Finished all backtest')
    
    def record_event(self, date_time, strategy, event, rule, price):
        new_row = {'date_time': date_time,
                   'strategy_name': strategy.name,
                   'position': strategy.position,
                   'event': event,
                   'rule': rule,
                   'price': price}
        self.trade_record = self.trade_record.append(new_row, ignore_index=True)
        
    def calculate_return(self):
        self.trade_record.loc[self.trade_record['event'] == OPEN_POSITION, 'trade_return'] = np.NaN
        
        for strategy in self.strategies:
            
            strategy_trade_record = self.trade_record[self.trade_record['strategy_name'] == strategy.name]
            strategy_trade_record['original_id'] = strategy_trade_record.index
            strategy_trade_record = strategy_trade_record.reset_index()
            
            for i in strategy_trade_record.index:
                
                if strategy_trade_record.at[i, 'event'] == CLOSE_POSITION:
                    date_time = strategy_trade_record.at[i, DATE_TIME]
                    entry_price = strategy_trade_record.at[i-1, 'price']
                    exit_price = strategy_trade_record.at[i, 'price']
                    trade_return = self.compute_return(entry_price, exit_price, strategy.position)
                    original_id = strategy_trade_record.at[i, 'original_id']
                    self.trade_record.at[original_id, 'trade_return'] = trade_return
                    
    @staticmethod
    def compute_return(entry_price, exit_price, position):
        if position == BULLISH:
            return (exit_price - entry_price) / entry_price
        elif position == BEARISH:
            return -1 * (exit_price - entry_price) / entry_price
        else:
            raise ValueError('[ERROR] Unknown Position')
    
    def gen_report(self):
        for strategy in self.strategies:
            df = self.trade_record[self.trade_record['strategy_name'] == strategy.name]
            
            occurrence = df[df['event'] == CLOSE_POSITION].shape[0]
            occ_profit = df[df['trade_return'] > 0].shape[0]
            occ_loss = df[df['trade_return'] < 0].shape[0]
            return_average = df['trade_return'].mean()
            return_std = df['trade_return'].std()
            
            new_row = {'strategy_name': strategy.name,
                       'occurrence': occurrence,
                       'occ_profit': occ_profit,
                       'occ_loss': occ_loss,
#                        'return_average': '{:.4%}'.format(return_average),
                       'return_average': return_average,
                       'return_std': return_std}
            self.report = self.report.append(new_row, ignore_index=True)
            
        self.gen_report_summary_row()
        print('[BACKTEST] Generated back test report')
        
    def gen_report_summary_row(self):
        new_row = {'strategy_name': 'summary',
                   'occurrence': self.report['occurrence'].sum(),
                   'occ_profit': self.report['occ_profit'].sum(),
                   'occ_loss': self.report['occ_loss'].sum(),
                   'return_average': '{:.4%}'.format(self.trade_record['trade_return'].mean()),
                   'return_std': self.trade_record['trade_return'].std()}
        self.report = self.report.append(new_row, ignore_index=True)
    
    def run(self):
        self.initialize()
        self.data_preprocess()
        self.compute_technical_indicator()
        self.back_test()
        self.gen_report()
        return

In [183]:
rule = Rule(
    name = 'basic', 
    indicators_param = [CCIParam(30, 0.015)],
    ref = {TI_CCI: TI_CCI},
    f = lambda data, ref, entry_i, i: data.at[i, ref[TI_CCI]] < -170)

open_position_param = OpenPositionParam(rules=[rule])

rule = Rule(
    name = 'basic', 
    indicators_param = [CCIParam(30, 0.015)],
    ref = {TI_CCI: TI_CCI},
    f = lambda data, ref, entry_i, i: data.at[i, ref[TI_CCI]] > 100)

rule_stop_loss = Rule(
    name = 'stop_loss', 
    indicators_param = [IndicatorParam(CLOSE)], 
    ref = {CLOSE: CLOSE}, 
    f = lambda data, ref, entry_i, i: (data.at[i, CLOSE] - data.at[entry_i, CLOSE]) < -300)

rule_stop_gain = Rule(
    name = 'stop_gain', 
    indicators_param = [IndicatorParam(CLOSE)], 
    ref = {CLOSE: CLOSE}, 
    f = lambda data, ref, entry_i, i: (data.at[i, CLOSE] - data.at[entry_i, CLOSE]) > 600)

close_position_param = ClosePositionParam(rules=[rule, rule_stop_loss])

CCI_bull_strategy = Strategy('CCI_bull_strategy', open_position_param, close_position_param, BULLISH)


rule = Rule(
    name = 'basic', 
    indicators_param = [CCIParam(30, 0.015)],
    ref = {TI_CCI: TI_CCI},
    f = lambda data, ref, entry_i, i: data.at[i, ref[TI_CCI]] > 100)

open_position_param = OpenPositionParam(rules=[rule])

rule = Rule(
    name = 'basic', 
    indicators_param = [CCIParam(30, 0.015)],
    ref = {TI_CCI: TI_CCI},
    f = lambda data, ref, entry_i, i: data.at[i, ref[TI_CCI]] < -170)

rule_stop_loss = Rule(
    name = 'stop_loss', 
    indicators_param = [IndicatorParam(CLOSE)], 
    ref = {CLOSE: CLOSE}, 
    f = lambda data, ref, entry_i, i: (data.at[i, CLOSE] - data.at[entry_i, CLOSE]) > 300)

rule_stop_gain = Rule(
    name = 'stop_gain', 
    indicators_param = [IndicatorParam(CLOSE)], 
    ref = {CLOSE: CLOSE}, 
    f = lambda data, ref, entry_i, i: (data.at[i, CLOSE] - data.at[entry_i, CLOSE]) < -600)

close_position_param = ClosePositionParam(rules=[rule, rule_stop_loss])

CCI_bear_strategy = Strategy('CCI_bear_strategy', open_position_param, close_position_param, BEARISH)



df = df = pd.read_csv('data/hsi_hourly.csv')
strategies = [CCI_bull_strategy, CCI_bear_strategy]
backtest = BackTest(df, strategies)
backtest.run()

[BACKTEST] Initializated
[CLEAN] 7 data items are filled
[BACKTEST] Finished data pre-process
[TechIndicatorEngine] Finished computation of CCI
[BACKTEST] Finished computation on technical indicators
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [184]:
backtest.report

,strategy_name,occurrence,occ_profit,occ_loss,return_average,return_std
0,CCI_bull_strategy,99,41,58,0.0536%,0.023348
1,CCI_bear_strategy,195,59,136,-0.0905%,0.025753
2,summary,294,100,194,-0.0420%,0.024939


In [8]:
class CCIBullishStrategyCreator():
    
    def __init__(self, period=20, ground=-100, sky=100, stop_gain=None, stop_loss=None):
        self.period = period
        self.ground = ground
        self.sky = sky
        self.stop_gain = stop_gain
        self.stop_loss = stop_loss
    
    def create(self):
       
        rule = Rule(
            name = 'basic', 
            indicators_param = [CCIParam(self.period, 0.015)],
            ref = {TI_CCI: TI_CCI},
            f = lambda data, ref, entry_i, i: data.at[i, ref[TI_CCI]] < self.ground)

        open_position_param = OpenPositionParam(rules=[rule])

        rule = Rule(
            name = 'basic', 
            indicators_param = [CCIParam(self.period, 0.015)],
            ref = {TI_CCI: TI_CCI},
            f = lambda data, ref, entry_i, i: data.at[i, ref[TI_CCI]] > self.sky)

        rules = [rule]

        if self.stop_gain is not None:
            rule_stop_gain = Rule(
                name = 'stop_gain', 
                indicators_param = [IndicatorParam(CLOSE)], 
                ref = {CLOSE: CLOSE}, 
                f = lambda data, ref, entry_i, i: (data.at[i, CLOSE] - data.at[entry_i, CLOSE]) / data.at[entry_i, CLOSE] > self.stop_gain)
            rules.append(rule_stop_gain)

        if self.stop_loss is not None:
            rule_stop_loss = Rule(
                name = 'stop_loss', 
                indicators_param = [IndicatorParam(CLOSE)], 
                ref = {CLOSE: CLOSE}, 
                f = lambda data, ref, entry_i, i: (data.at[i, CLOSE] - data.at[entry_i, CLOSE]) / data.at[entry_i, CLOSE] < self.stop_loss)
            rules.append(rule_stop_loss)

        close_position_param = ClosePositionParam(rules=rules)

        return Strategy('CCI_bull_strategy', open_position_param, close_position_param, BULLISH)

In [9]:
class CCIBearishStrategyCreator():
    
    def __init__(self, period=20, ground=-100, sky=100, stop_gain=None, stop_loss=None):
        self.period = period
        self.ground = ground
        self.sky = sky
        self.stop_gain = stop_gain
        self.stop_loss = stop_loss
    
    def create(self):
       
        rule = Rule(
            name = 'basic', 
            indicators_param = [CCIParam(self.period, 0.015)],
            ref = {TI_CCI: TI_CCI},
            f = lambda data, ref, entry_i, i: data.at[i, ref[TI_CCI]] > self.sky)

        open_position_param = OpenPositionParam(rules=[rule])

        rule = Rule(
            name = 'basic', 
            indicators_param = [CCIParam(self.period, 0.015)],
            ref = {TI_CCI: TI_CCI},
            f = lambda data, ref, entry_i, i: data.at[i, ref[TI_CCI]] < self.ground)

        rules = [rule]

        if self.stop_gain is not None:
            rule_stop_gain = Rule(
                name = 'stop_gain', 
                indicators_param = [IndicatorParam(CLOSE)], 
                ref = {CLOSE: CLOSE}, 
                f = lambda data, ref, entry_i, i: (data.at[i, CLOSE] - data.at[entry_i, CLOSE]) / data.at[entry_i, CLOSE] < self.stop_gain)
            rules.append(rule_stop_gain)

        if self.stop_loss is not None:
            rule_stop_loss = Rule(
                name = 'stop_loss', 
                indicators_param = [IndicatorParam(CLOSE)], 
                ref = {CLOSE: CLOSE}, 
                f = lambda data, ref, entry_i, i: (data.at[i, CLOSE] - data.at[entry_i, CLOSE]) / data.at[entry_i, CLOSE] > self.stop_loss)
            rules.append(rule_stop_loss)

        close_position_param = ClosePositionParam(rules=rules)

        return Strategy('CCI_bear_strategy', open_position_param, close_position_param, BEARISH)

In [12]:
cci_bullish_strategy_creator = CCIBullishStrategyCreator(
    period = 20,
    ground = -150, 
    sky = 150, 
    stop_gain = 0.05, 
    stop_loss = -0.05)
CCI_bull_strategy = cci_bullish_strategy_creator.create()

cci_bearish_strategy_creator = CCIBearishStrategyCreator(
    period = 20,
    ground = 150, 
    sky = -150, 
    stop_gain = -0.03, 
    stop_loss = 0.05)
CCI_bear_strategy = cci_bearish_strategy_creator.create()

df = pd.read_csv('data/hsi_hourly.csv')
strategies = [CCI_bull_strategy, CCI_bear_strategy]
backtest = BackTest(df, strategies)
# backtest.run()

backtest.initialize()
backtest.data_preprocess()
backtest.compute_technical_indicator()

backtest.data

# backtest.report

[BACKTEST] Initializated
[CLEAN] 7 data items are filled
[BACKTEST] Finished data pre-process
[TechIndicatorEngine] Finished computation of CCI
[BACKTEST] Finished computation on technical indicators


,date_time,open,high,low,close,CCI
0,2012-01-30 09:15,20518.00,20518.00,20363.47,20414.82,NaN
1,2012-01-30 10:15,20407.75,20407.75,20336.88,20394.78,NaN
2,2012-01-30 11:15,20385.29,20443.24,20374.04,20401.32,NaN
3,2012-01-30 13:15,20370.87,20390.54,20318.81,20342.81,NaN
4,2012-01-30 14:15,20339.58,20339.58,20220.33,20258.05,NaN
...,...,...,...,...,...,...
15022,2020-11-06 11:15,25688.27,25696.39,25619.07,25649.51,60.251818
15023,2020-11-06 12:15,25640.23,25653.78,25604.68,25627.03,52.284047
15024,2020-11-06 13:15,25627.24,25653.45,25540.98,25637.72,45.158915
15025,2020-11-06 14:15,25641.66,25708.36,25620.41,25690.42,50.206255


In [13]:
backtest.data.to_csv('data/hsi_hourly_cleaned.csv', index=False)

In [ ]:
backtest.trade_record

In [ ]:
backtest.report

In [ ]:
display_df(backtest.trade_record)

In [17]:
class FieldToTune():
    def __init__(self, name, low_bound, up_bound):
        self.name = name
        self.low_bound = low_bound
        self.up_bound = up_bound

In [49]:
class StrategyTuner():
    
    def __init__(self, data, strategy_creator, fields_to_tune):
        self.record = pd.DataFrame()
        self.report = pd.DataFrame()
        self.num_trial = 1000
        self.num_top_result = 100
        
        self.data = data
        self.strategy_creator = strategy_creator
        self.strategy = None
        self.fields_to_tune = fields_to_tune
        self.backtest = None
        
        self.file_path = None
        
        self.initialize()
    
    def initialize(self):
        self.update_strategy()
        self.file_path = 'data/' + self.strategy.name + '_param_tune_result.csv'
        self.initialize_backtest()
        self.initialize_record()
        self.initialize_report()
        
    def initialize_backtest(self):
        self.backtest = BackTest(self.data, [self.strategy])
        self.backtest.initialize()
        self.backtest.data_preprocess()
        self.backtest.compute_technical_indicator()
        
    def initialize_record(self):
        self.record = pd.DataFrame()
        for field in self.fields_to_tune:
            self.record[field.name] = None
            
    def initialize_report(self):
        if os.path.isfile(self.file_path):
            self.report = pd.read_csv(self.file_path)
            print('[StrategyTuner] File found: {}'.format(self.file_path))
        else:
            self.report = pd.DataFrame()
            self.report['strategy_name'] = None
            self.report['occurrence'] = None
            self.report['occ_profit'] = None
            self.report['occ_loss'] = None
            self.report['return_average'] = None
            self.report['return_std'] = None
            self.report['simulation_timestamp'] = None
            for field in self.fields_to_tune:
                self.report[field.name] = None
            print('[StrategyTuner] File not found: {}. Created new file.'.format(self.file_path))
    
    def simulate(self):
        self.update_strategy()
        self.backtest.strategies = [self.strategy]
        self.backtest.initialize()
        self.backtest.back_test()
        self.backtest.gen_report()
    
    def update_strategy(self):
        for field in self.fields_to_tune:
            self.strategy_creator.__dict__[field.name] = random.uniform(field.low_bound, field.up_bound)
        self.strategy = self.strategy_creator.create()
    
    def record_simulation(self):
        new_row = {}
        for col in self.record.columns:
            new_row[col] = self.strategy_creator.__dict__[col]
        self.record = self.record.append(new_row, ignore_index=True)
        self.save_top_result(new_row)
    
    def save_top_result(self, param_config):
        new_row = dict()
        new_row.update(dict(self.backtest.report.iloc[0]))
        new_row.update(param_config)
        new_row['simulation_timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.report = self.report.append(new_row, ignore_index=True)
        self.report = self.report.sort_values('return_average', ascending=False)
        self.report = self.report[:self.num_top_result]
        self.report = self.report.reset_index(drop=True)
        self.report.to_csv(self.file_path, index=False)
    
    def run_simulation(self):
        for i in range(self.num_trial):
            self.simulate()
            self.record_simulation()
            print(i)
            print('best return = {:.4%}, this return = {:.4%}'.format(self.report.at[0, 'return_average'], self.backtest.report.at[0, 'return_average']))

In [51]:
df = pd.read_csv('data/hsi_hourly.csv')
cci_bulld_strategy_creator = CCIBullishStrategyCreator()
fields_to_tune = (
    FieldToTune(name='ground', low_bound=-200, up_bound=-50),
    FieldToTune(name='sky', low_bound=50, up_bound=200),
    FieldToTune(name='stop_gain', low_bound=0.03, up_bound=0.1),
    FieldToTune(name='stop_loss', low_bound=-0.02, up_bound=-0.1)
)

strategy_tuner = StrategyTuner(df, cci_bulld_strategy_creator, fields_to_tune)
strategy_tuner.run_simulation()

[BACKTEST] Initializated
[CLEAN] 7 data items are filled
[BACKTEST] Finished data pre-process
[TechIndicatorEngine] Finished computation of CCI
[BACKTEST] Finished computation on technical indicators
[StrategyTuner] File found: data/CCI_bull_strategy_param_tune_result.csv
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
0
best return = 0.8319%, this return = -0.2890%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
1
best return = 0.8319%, this return = 0.8143%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
2
best return = 0.8319%, this return = 0.4429%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
3
best return = 0.8319%, this return = -0.1751%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
4
best return = 0.8319%, this return = 0.5266%
[BACKTEST]

[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
55
best return = 0.8319%, this return = -0.2164%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
56
best return = 0.8319%, this return = -0.1533%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
57
best return = 0.8319%, this return = 0.0146%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
58
best return = 0.8319%, this return = -0.0494%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
59
best return = 0.8319%, this return = -0.1134%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
60
best return = 0.8319%, this return = -0.0316%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
61
best return = 0.8319%, this return = -0.1702%
[BACKTEST] 

[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
112
best return = 0.8319%, this return = 0.0785%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
113
best return = 0.8319%, this return = 0.1768%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
114
best return = 0.8319%, this return = 0.2220%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
115
best return = 0.8319%, this return = 0.0617%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
116
best return = 0.8319%, this return = 0.5168%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
117
best return = 0.8319%, this return = 0.1249%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
118
best return = 0.8319%, this return = -0.3612%
[BACKTEST

[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
169
best return = 0.8319%, this return = 0.3377%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
170
best return = 0.8319%, this return = -0.4363%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
171
best return = 0.8319%, this return = -0.1201%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
172
best return = 0.8319%, this return = 0.1288%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
173
best return = 0.8319%, this return = 0.2302%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
174
best return = 0.8319%, this return = -0.0465%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
175
best return = 0.8319%, this return = 0.0997%
[BACKTE

[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
226
best return = 0.8319%, this return = -0.2110%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
227
best return = 0.8319%, this return = 0.0879%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
228
best return = 0.8319%, this return = -0.0170%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
229
best return = 0.8319%, this return = 0.0690%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
230
best return = 0.8319%, this return = -0.1756%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
231
best return = 0.8319%, this return = -0.5049%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
232
best return = 0.8319%, this return = 0.2080%
[BACKT

...

[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
739
best return = 0.8319%, this return = -0.1009%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
740
best return = 0.8319%, this return = -0.2003%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
741
best return = 0.8319%, this return = 0.1184%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
742
best return = 0.8319%, this return = 0.0753%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
743
best return = 0.8319%, this return = 0.3548%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
744
best return = 0.8319%, this return = 0.0985%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
745
best return = 0.8319%, this return = -0.0791%
[BACKTE

[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
796
best return = 0.8319%, this return = 0.0229%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
797
best return = 0.8319%, this return = -0.1207%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
798
best return = 0.8319%, this return = -0.1050%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
799
best return = 0.8319%, this return = -0.0127%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
800
best return = 0.8319%, this return = 0.2087%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
801
best return = 0.8319%, this return = 0.3367%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
802
best return = 0.8319%, this return = 0.1241%
[BACKTE

[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
853
best return = 0.8319%, this return = 0.0542%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
854
best return = 0.8319%, this return = -0.1272%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
855
best return = 0.8319%, this return = -0.1491%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
856
best return = 0.8319%, this return = 0.2552%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
857
best return = 0.8319%, this return = 0.2016%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
858
best return = 0.8319%, this return = 0.1790%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
859
best return = 0.8319%, this return = -0.1134%
[BACKTE

[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
910
best return = 0.8319%, this return = 0.0163%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
911
best return = 0.8319%, this return = -0.3266%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
912
best return = 0.8319%, this return = 0.5205%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
913
best return = 0.8319%, this return = -0.1160%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
914
best return = 0.8319%, this return = 0.7071%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
915
best return = 0.8319%, this return = 0.1575%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
916
best return = 0.8319%, this return = 0.0196%
[BACKTES

[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
967
best return = 0.8319%, this return = -0.1478%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
968
best return = 0.8319%, this return = 0.3983%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
969
best return = 0.8319%, this return = -0.0569%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
970
best return = 0.8319%, this return = 0.3150%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
971
best return = 0.8319%, this return = -0.0141%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
972
best return = 0.8319%, this return = -0.3038%
[BACKTEST] Initializated
[BACKTEST] Finished all backtest
[BACKTEST] Generated back test report
973
best return = 0.8319%, this return = 0.0584%
[BACKT

In [39]:
strategy_tuner.report

,strategy_name,occurrence,occ_profit,occ_loss,return_average,return_std,ground,sky,stop_gain,stop_loss,Unnamed: 10,simulation_timestamp
0,CCI_bull_strategy,70,54,16,0.008319,0.044366,-194.510074,195.529816,0.040471,-0.079177,NaN,NaN
1,CCI_bull_strategy,70,45,25,0.007811,0.052087,-174.119728,198.922367,0.098470,-0.060250,NaN,NaN
2,CCI_bull_strategy,65,46,19,0.007808,0.053541,-184.734346,195.002974,0.086097,-0.077474,NaN,NaN
3,CCI_bull_strategy,68,49,19,0.007775,0.048315,-196.158518,171.606433,0.085192,-0.094779,NaN,NaN
4,CCI_bull_strategy,82,61,21,0.007390,0.052138,-96.677099,198.521348,0.081728,-0.091980,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CCI_bull_strategy,121,76,45,0.002555,0.041444,-84.942749,192.779923,0.049761,-0.049553,NaN,NaN
96,CCI_bull_strategy,138,85,53,0.002551,0.039692,-74.045119,187.299266,0.052853,-0.047357,NaN,NaN
97,CCI_bull_strategy,203,126,77,0.002526,0.033567,-69.430688,154.722002,0.096622,-0.032392,NaN,11/08/2020 16:52:50
98,CCI_bull_strategy,69,46,23,0.002520,0.042141,-198.917220,163.709005,0.071758,-0.060333,NaN,NaN


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.hist(list(df[TI_CCI]), bins=200)
plt.title('CCI Distribution')
plt.xlabel('CCI')
plt.ylabel('Frequency')
plt.xlim(-500, 500)
plt.xticks(np.arange(-500, 501, 100))
plt.grid(True)
plt.show()